# Representación semántica y búsqueda
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

## Usando la nube de puntos

La representación de nube de puntos puede usarse directamente sin realizar prototipos, sin embargo, es posible que dada su complejidad, dimensión intrinseca, tenga poca utilidad en grandes volumenes de información. Por ejemplo, es posible utilizar la distancia de [Hausdorff](https://en.wikipedia.org/wiki/Hausdorff_distance); que entre dos nubes de puntos (documentos para este caso) esta definida como sigue:

$$ H_d(U, V) = \max \left\{ \max_{u \in U} \min_{x \in V} d(u, x), \max_{v \in V} \min_{x \in U} d(v, x) \right\} $$

Donde $\textsf{nn}$ encuentra el vecino cercano del primer argumento en el segundo. Podemos ver la intuición de esta distancia, analizando las partes de la expresión. 
La diferencia entre ambos conjuntos se representa con el máximo de las distancias mínimas. Note que esta parte es idéntica a resolver una búsqueda de vecinos cercanos. La parte más externa de la expresión se repite para los dos conjuntos para preservar la simétria de la función.

De la misma forma, es posible obtener variaciones de interés como considerar todas las distancias cercanas en lugar de solo las máximas, lo cual puede reducir formas caprichosas de _outlayers_.

$$ H^+_d(U, V) = \max \left\{ \sum_{u \in U} \min_{x \in V} d(u, x), \sum_{v \in V} \min_{x \in U} d(v, x) \right\}$$

La información local de una representación de bolsa de palabras también puede ser aprovechada, por ejemplo, añadiendo información de $\textsf{TF}$

$$ H^\textsf{TF}_d(U, V) = \max \left\{ \sum_{u \in U} {\textsf{TF}(u, U) \min_{x \in V} d(u, x)}, \sum_{v \in V} {\textsf{TF}(v, V) \min_{x \in U} d(v, x)} \right\} $$

# Notas adicionales
La versión de $H_d$ que usa la probabilidad de término $\textsf{pt}$ es similar a la aproximación 2 listada en [@KSKW2015]. Los prototipos son similares a la aproximación 1 del mismo artículo. En el artículo se presenta la función de distancia _word mover's distance_ (WMD) que es una adecuación de la Earths mover's distance (EMD) [PW2009]. EMD es la solución óptima a un problema de transportación, la WMD es la adaptación teniendo en cuenta embeddings de palabras.

# Ejemplo


In [1]:
using Pkg
Pkg.activate(".")
!isfile("Manifest.toml") && Pkg.add([
    PackageSpec(name="SimilaritySearch", version="0.9"),
    PackageSpec(name="TextSearch", version="0.13"),
    PackageSpec(name="Plots"),
    PackageSpec(name="KNearestCenters"),
    PackageSpec(name="HypertextLiteral"),
    PackageSpec(name="CSV"),
    PackageSpec(name="DataFrames"),
    PackageSpec(name="Word2Vec"),
    PackageSpec(name="Embeddings"),
])

using SimilaritySearch, TextSearch, Plots, Word2Vec, KNearestCenters, LinearAlgebra, HypertextLiteral, CSV, DataFrames, Base64, Random
using Embeddings: load_embeddings
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
]status

      Status `~/IR-2022/Unidades/Project.toml`
  [944b1d66] CodecZlib v0.7.0
  [ac1192a8] HypertextLiteral v0.9.4
  [b20bd276] InvertedFiles v0.4.2 `../../Research/InvertedFiles.jl`
  [682c06a0] JSON v0.21.3
  [4dca28ae] KNearestCenters v0.7.1
  [8ef0a80b] Languages v0.4.3
  [eb30cadb] MLDatasets v0.7.3
  [053f045d] SimilaritySearch v0.9.4 `../../Research/SimilaritySearch.jl`
  [fb8f903a] Snowball v0.1.0
  [2913bbd2] StatsBase v0.33.18
  [7f6f6c8a] TextSearch v0.12.7 `../../Research/TextSearch.jl`
  [c64b6f0f] Word2Vec v0.5.3


In [3]:
function word2vec_embeddings(textconfig, corpus, embfile; dim=32, w2vtmp=tempfile(pwd()))
    isfile(embfile) && return
    tcorpus = tokenize_corpus(textconfig, corpus)
    voc = Vocabulary(textconfig, tcorpus)
    fvoc = filter_tokens(voc) do t
        5 <= t.ndocs <= 2000
    end
    valid = Set(fvoc.token)

    open(w2vtmp, "w") do f
        for tokens in tcorpus
            tokens = filter!(t -> t in valid, tokens)
            if length(tokens) > 7 # minimum number of tokens in a message
                println(f, join(tokens, ' '))
            end
        end
    end

    word2vec(w2vtmp, embfile, size=dim, iter=15, threads=Threads.nthreads()-1)
end


word2vec_embeddings (generic function with 1 method)

In [ ]:
textconfig = TextConfig(; nlist=[1], qlist=[], slist=[], lc=true,
    group_usr=true, group_url=true, group_num=true, del_diac=true, del_punc=true)

filename = "../data/spanish-news-in-twitter.csv"
embfile = filename * ".w2v.vec"
@time D = CSV.read(filename, DataFrame; delim=',', missingstring="XXXX")
@time D[:, :text] = String.(base64decode.(D.text))
rm(embfile)
word2vec_embeddings(textconfig, D.text, embfile; dim=64, w2vtmp=joinpath(pwd(), "w2v.tmp"))
E = load_embeddings(FastText_Text, embfile)


In [ ]:
T = let corpus=D.text
    voc = Vocabulary(textconfig, corpus; thesaurus=E.vocab)
    model = VectorModel(BinaryGlobalWeighting(), TpWeighting(), voc)
    (; textconfig, model, voc)
end

In [ ]:
cloud(text, E, T) = vectorize(T.model, T.textconfig, text; normalize=false)
cloud_corpus(corpus, E, T) = vectorize_corpus(T.model, T.textconfig, corpus; normalize=false)

In [ ]:
struct AWMD <: SemiMetric
    X::MatrixDatabase
    results::Vector{KnnResult}
    dcos::NormalizedCosineDistance
end

AWMD(X) = AWMD(X, [KnnResult(1) for _ in 1:Threads.nthreads()], NormalizedCosineDistance())

const PI_2 = convert(Float32, pi / 2)

function nn_(wmd, res, dcos, u, V)
    res = reuse!(res)

    for vid in keys(V)
        v = wmd.X[vid]
        push!(res, vid, evaluate(dcos, u, v))
    end
    
    minimum(res)
end

function onesidewmd(wmd, res, dcos, U, V)
    d = zero(Float32)
    @fastmath for (uid, w) in U
        if !haskey(V, uid) # evaluates to zero
            d = max(d, w * nn_(wmd, res, dcos, wmd.X[uid], V))
        end
    end
    
    d
end

function SimilaritySearch.evaluate(wmd::AWMD, U::Dict, V::Dict)
    length(U) == 1 && haskey(U, 0) && return PI_2
    length(V) == 1 && haskey(V, 0) && return PI_2
    res = wmd.results[Threads.threadid()]
    dcos = wmd.dcos

    max(onesidewmd(wmd, res, dcos, U, V), onesidewmd(wmd, res, dcos, V, U))
end


In [ ]:
db = VectorDatabase(cloud_corpus(D.text, E, T))
for c in eachcol(E.embeddings)
    normalize!(c)
end

dist = AWMD(MatrixDatabase(E.embeddings))

In [ ]:
#=recall = 0.8
index = SearchGraph(; dist, db, verbose=true)
#index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
index!(index)
optimize!(index, MinRecall(recall))
=#

In [ ]:
index = ParallelExhaustiveSearch(; db, dist);

In [ ]:
Threads.nthreads()

In [ ]:
#@time res, _ = search(ex, db[111], KnnResult(10))
#=@time res, _ = search(ex, cloud("covid en México", E, T), KnnResult(10))

for (id_, dist_) in res
    display(@htl """<div style="padding: 0.5em;">
        <span style="width: 25%; margin: 0.25em;"> $(id_ => round(dist_; digits=3))</span>
        <span style="background-color: rgb(120, 60, 60); margin: 0.2em;">$(D.text[id_])</span>
    </div>""")
end
=#

### Búsqueda y presentación de los resultados

In [ ]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = cloud(qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.screen_name[id])</td><td> $(D.text[id])</td> </tr>")
    end

    display(@htl """<h2>resultados para "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


In [ ]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


In [ ]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")

for i in 1:3
    for qid in rand(1:length(D.text))
        search_and_display(index, D.text[qid], 7, D, E, T)
    end
end

# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Defina $H^{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.- Defina $H^{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.
- Implemente $H^\textsf{TF}_d$ y $H^\textsf{pt}_d$ de manera secuencial y con un índice métrico. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. El reporte deberá contener un ensayo de [@KSKW2015] como introducción. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
